<a href="https://colab.research.google.com/github/Davydkova/Davydkova-Data_analyst/blob/main/Pet__%D0%9C%D0%B5%D1%80%D0%BA%D1%83%D1%80_%D0%90%D0%B2%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ЗАКАЗЧИК : Меркур Авто

ЦЕЛЬ: Определить рыночные сегменты Меркур Авто предложить маркетинговый план.



Описание данных (продажи автомобилей в Казахстане 2019 год)


*   Год – год продажи (2019)
*   Месяц – месяц продажи (январь - сентябрь)
*   Компания – название автоцентра
*   Бренд – название продаваемой марки автомобиля
*   Модель – название модели автомобиля
*   Модификация – модификация модели автомобиля
*   Год выпуска – год производства автомобиля
*   Страна-производитель – страна, где произведен автомобиль
*   Вид топлива – бензин, дизель, электричество, гибрид
*   Объём двиг л – объем двигателя автомобиля в литрах
*   Коробка передач – тип коробки переключения передач
*   Сегмент – сегмент, к которому относится авто (удаляем)
*   Тип привода – в итоге оставляем RWD – задний привод, FWD – передний привод,
4WD – полный привод, 2WD – все остальное (подключаемый полный привод и где
нет четкого указания передний или задний это привод)
*   Регион – регион продажи
*   Тип клиента – юридическое или физическое лицо
*   Форма расчета – наличный и безналичный расчет

*   Количество – количество автомобилей в заказе
*   Цена USD – цена автомобиля

*   Продажа USD – цена заказа (цена авто умноженная на количество и за вычетом
скидок если есть)
*   Область – область продажи
*   Сегментация 2013 – сегмент автомобиля актуальный
*   Класс 2013 – класс автомобиля актуальный
*   Сегментация Eng – английская сегментация
*   Локализация производства – удаляем (совпадает со страной производителем)






















# Загрузка данных

In [113]:
!pip install -q yadisk
#!pip install -q pywaffle
!pip install -q squarify

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import requests
from urllib.parse import urlencode
import squarify
import plotly.graph_objects as go
import plotly.express as px
import statistics as st
import math
from collections import Counter

%matplotlib inline

warnings.filterwarnings('ignore')

#опции: количество отображаемых столбцов, количество знаков десятых разрядов
pd.set_option('display.max_columns',30)
pd.set_option('display.float_format', '{:2f}'.format)

In [115]:
try:
    df= pd.read_csv('auto_kz_2019_датасет.csv', sep=';')
except:
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/d/ys6DfbYNFlOXDA'
    # получаем url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # загружаем файл в df
    download_response = requests.get(download_url)
    df = pd.read_csv(download_url, sep=';')

In [116]:
start_shape = df.shape
print(f'Итого в данных {start_shape[0]} наблюдений с {start_shape[1]} признаками')

Итого в данных 39966 наблюдений с 25 признаками


In [117]:
# общая информация
display(df.sample(5))

df.info()
df.columns

df.describe().T
print('В датасете {} полных дубликатов'.format(df.duplicated().sum()))

#соотношение пропущенных значений по убыванию
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values(by=['percent_missing'], ascending=False)

,Год,Месяц,Компания,Бренд,Модель,Модификация,Год выпуска,Страна-производитель,Вид топлива,"Объём двиг, л,",Коробка передач,Тип привода,Сегмент,Регион,Наименование дилерского центра,Тип клиента,Форма расчета,Количество,"Цена, USD","Продажа, USD",Область,Сегментация 2013,Класс 2013,Сегментация Eng,Локализация производства
23176,2019,Сентябрь,Ravon Motors Kazakstan,Ravon,Nexia R3,Optimum АT,2019,Республика Казахстан,Бензин,"1,5",АT,2WD,NaN,Кызылорда,Allur Auto Кызылорда,Физ. Лицо,кредит,1.000000,"9884,695773","9884,695773",Кызылординская область,Легковые автомобили,B класс,B,Локальное производство
14702,2019,Июнь,Allur Auto,Jac,S5,Intelligent 2.0T MT6,2019,Китай,NaN,NaN,NaN,NaN,NaN,Костанай,Allur Auto Almaty,Юр. Лицо,безналичный,1.000000,"11245,55322","11245,55322",Костанайская область,Внедорожники,Субкомпактные SUV,K1,Импорт
25598,2019,Март,БИПЭК АВТО,Skoda,Rapid,NaN,2019,Республика Казахстан,Бензин,"1,6",6 АТ,Передний,B класс,Атырау,БИПЭК АВТО Атырау,NaN,NaN,1.000000,13900,13900,Атырауская область,Легковые автомобили,B класс,B,Локальное производство
31343,2019,Апрель,Toyota Motor Kazakhstan,Toyota,Corolla,57,2019,Турция,Бензин,"2,7",AT,4WD,C класс,Костанай,Тойота Центр Костанай,Физ. Лицо,NaN,1.000000,"22531,08605","22531,08605",Костанайская область,Легковые автомобили,C класс,C,Импорт
31221,2019,Март,Toyota Motor Kazakhstan,Toyota,Corolla,54,2 019,Турция,Бензин,2.7,AT,4WD,C класс,Уральск,Тойота Центр Орал,Физ. Лицо,NaN,1.000000,"42413,7931","42413,7931",Западно-Казахстанская область,Легковые автомобили,C класс,C,Импорт


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Год                             39966 non-null  int64  
 1   Месяц                           39966 non-null  object 
 2   Компания                        39966 non-null  object 
 3   Бренд                           39966 non-null  object 
 4   Модель                          39966 non-null  object 
 5   Модификация                     36375 non-null  object 
 6   Год выпуска                     39465 non-null  object 
 7   Страна-производитель            39966 non-null  object 
 8   Вид топлива                     36826 non-null  object 
 9   Объём двиг, л,                  35708 non-null  object 
 10  Коробка передач                 36711 non-null  object 
 11  Тип привода                     35677 non-null  object 
 12  Сегмент                         

,column_name,percent_missing
Форма расчета,Форма расчета,64.875144
Тип клиента,Тип клиента,17.632488
Сегмент,Сегмент,16.916879
Тип привода,Тип привода,10.731622
"Объём двиг, л,","Объём двиг, л,",10.654056
Модификация,Модификация,8.985137
Коробка передач,Коробка передач,8.144423
Вид топлива,Вид топлива,7.856678
Год выпуска,Год выпуска,1.253566
Количество,Количество,0.015013



 Общий вывод

1)  Выявлены неполные данные в 10 столбцах из 25, 40 % от общего количества столбцов,
необходима обработка этих данных:
- заполнение при возможности статистическими значениями ( если выявлена корреляция )
- возможность поставить "заглушки" в пропусках,
- заменить данными из внешних источников,
- провести анализ пропущенных значений в данных при пересечении с другими критериями объекта.  
- удаление столбцов с большим количеством пропусков, которые корректному восстановлению не подлежат

2) Типы данных нуждаются в предобработке

3) Необходима работа с дубликатами. Выявлено 18698 дубликатов. 46 % от данных. Появление данных дублей связано с отсутствует id сделки

Чем больше данных мы сохраним от изначального объема, тем более качественный анализ сможем получить .


## Предобработка данных

In [118]:
#замена наименований столбцов
сolumns_old = df.columns.tolist()

columns_new = ['year','month','company','brend','model','modification','year_release',
               'manufacturing_country','type_of_fuel','volume_engine','transmission','type_of_drive',
              'segment','region','diler','client_type','form_payment','count','price','sale',
              'area','segment_2013','class_2013','segment_eng','localization_production']

change_col_dict = {i:k for i, k in zip(сolumns_old, columns_new )}

df = df.rename(columns=change_col_dict)

In [119]:
df = df.drop (columns = ['modification','form_payment','segment',
                         'segment_eng','localization_production'])

"Тип оплаты" содержит 64,8 % пропусков, данные восстановить не представляется возможным.
"Локализация производства" восстанавливается по признаку страна производитель .
Удаляем "сегменты", "модифицикация машин" ( у всех диллеров разные кодировки, связи не выявлено в данных )

## ПРЕДОБРАБОТКА. Количественные признаки

#### Volume_engine | Объем двигателя

In [120]:
df.volume_engine.unique()

array(['1,4', '2', '3', '4,4', '6,6', '1,5', '1,596', '2,4', '1,7',
       '26,7', '25,7', '24,7', '23,7', '22,7', '21,7', '20,7', '19,7',
       '18,7', '17,7', '16,7', '15,7', '14,7', '13,7', '12,7', '11,7',
       '10,7', '9,7', '8,7', '7,7', '6,7', '5,7', '4,7', '3,7', '2,7',
       '6,2', '1,4 Turbo', '2,5', '1,6', nan, '2,8', '4,3', '4,3,',
       '1,6 MPI', '2,0 MPI', '2,4 GDI', '2,4 MPI', '2,5 CRDI VGT',
       '2,5 CRDI WGT', '3,9', '7,5', '12,3', '1,6 T-GDI', '2,0 CRDI',
       '2.0', '3,5', '5,6', '5,2', '3,0 L', '1.6', '1.5', '5', '1,598',
       '1,248', '1,998', '2,359', '1,999', '3,342', '1,591', '3,47',
       '1,69', '1,774', '2.5', '2.7 ', '3.5', '2.7', '4.6', '4,6', '5.7',
       '1,8', '10,5', '4', '5,5', '12', '12,8', '11', '2,2', '1,2',
       '1,485', 'MT', 'AT', '0', '13', '1,33', '#Н/Д', '4.0', '2.4G',
       '2,4G', '2.8', '2,693', '2,0', '3,8', '3.8', '1,2T', '3,6', '7,6',
       '4,9', '2.0h', '2,9', '400 Л.С.', '4,98 L,', '4,98', '88 KWH',
       '8,4 L,', 

In [121]:
# замена значений, предобработка
df.loc[df.volume_engine == '400 Л.С.', 'volume_engine'] = 2
df.loc[df.volume_engine == '88 KWH', 'volume_engine'] = -1
df.volume_engine = df.volume_engine.str.replace(',','.')

df.volume_engine = df.volume_engine.replace({'1,6':'1.6','1.4 Turbo':'1.4','3/0':'3.0',
                                            '1.6 MPI':'1.6','2.0 MPI':'2.0','4.3.':'4.3',
                                            '2.4 GDI':'2.4', '2.4 MPI':'2.4',
                                            '2.5 CRDI VGT':'2.5', '2.5 CRDI WGT':'2.5',
                                            '1.6 T-GDI':'1.6', '2.0 CRDI':'2.0', '3.0 L':'3/0',
                                            '2.4G':'2.4','1.2T':'1.2','2.0h':'2.0',
                                            '4.98 L.':'4.98', '4.98':'4.98', '8.4 L.':'8.4', '6.7L':'6.7'})

<div class="alert alert-block alert-warning"> <b>Reviewer's comment</b> <a
class="tocSkip"></a>
    
Здесь хотелось бы чтобы было аргументирована замена 88 KWH на -1 Кроме того для 400 лс если ты посмотришь вид топлива то увидишь что это электроавтомобиль Соответсвенно к нему объем двигателя тоже не применим Для замены значений я бы тебе посоветовала не составлять словарь соответствий а использовать regex оставляя только числа
</div>

In [122]:
#cрез данных
engine_query = df.query (" volume_engine == ['MT','AT'] ")
engine_query_not = df.query (" volume_engine not in ['MT','AT'] ")

print(f'Количество строк {engine_query.shape[0]}')

engine_query[['type_of_fuel','transmission','volume_engine','type_of_drive']].sample(3)

Количество строк 34


,type_of_fuel,transmission,volume_engine,type_of_drive
23729,2,4WD,AT,4WD
23753,2,4WD,AT,4WD
23724,2,4WD,AT,4WD


In [123]:
# смещение данных
engine_query ['transmission'] = engine_query ['volume_engine']
engine_query ['volume_engine'] = engine_query ['type_of_fuel']
engine_query ['type_of_fuel'] = np.NaN

# слияние датасетов
df = pd.concat ((engine_query,engine_query_not))
df.shape

(39966, 20)

In [124]:
# общая сумма пропусков
df.volume_engine.isna().mean()

0.10686583596056648

10,6 % пропусков в столбце volume_engine

In [125]:
# топ 10 пропусков в пересечении с моделью
df[df.volume_engine.isna()].model.value_counts().nlargest(10)

Duster      650
Sandero     435
Nexia R3    361
Logan       320
65115       259
Kaptur      230
43118       188
S3          175
3302        127
S5          117
Name: model, dtype: int64

In [126]:
# cписок моделей с пропусками в двигателе
engine_nans = df[df.volume_engine.isna()].model.value_counts().nlargest(10).reset_index()['index'].tolist()
df[df.model.isin(engine_nans)].groupby('model')['volume_engine'].value_counts()

model     volume_engine
3302      2.7               866
          2.8                 3
          4.3                 2
Duster    2                  42
          1,6                 9
          1.6                 6
Nexia R3  1.5              1202
          1.485             136
S3        1.6              1109
          1.5                 2
S5        2                 243
          2.0                87
Sandero   0                   3
          1,6                 2
          1.6                 1
Name: volume_engine, dtype: int64

In [127]:
df['volume_engine'] = df.groupby(['model'], sort=False)['volume_engine'].apply(lambda x: x.fillna(x.mode()[0])
                                                                       if len(x.mode()) >= 1 else x)

In [128]:
df['volume_engine'].isna().mean()

0.045538707901716456

На оставшиеся пропуски в столбце тип двигателя принято решение поставить "заглушки" , так как дальнейшей связи с признаками не выявлено.

При согласовании с заказщиком возможно допущение : замена на часто встречающееся значения типа двигателя в зависимости от бреда, и дальнейшая категоризация данного признака при анализе.

In [129]:
df.query('volume_engine == "#Н/Д"')

,year,month,company,brend,model,year_release,manufacturing_country,type_of_fuel,volume_engine,transmission,type_of_drive,region,diler,client_type,count,price,sale,area,segment_2013,class_2013
31148,2019,Февраль,Toyota Motor Kazakhstan,Toyota,Corolla,2019,Турция,Бензин,#Н/Д,#Н/Д,#Н/Д,Нур-Султан,Тойота Центр Астана,Физ. Лицо,1.000000,"42316,46043","42316,46043",г.Нур-Султан,Легковые автомобили,C класс
31149,2019,Февраль,Toyota Motor Kazakhstan,Toyota,Corolla,2019,Турция,Бензин,#Н/Д,#Н/Д,#Н/Д,Нур-Султан,Тойота Центр Астана,Физ. Лицо,1.000000,"42316,46043","42316,46043",г.Нур-Султан,Легковые автомобили,C класс
32670,2019,Апрель,Toyota Motor Kazakhstan,Toyota,HILUX DC,2019,Таиланд,Бензин,#Н/Д,#Н/Д,#Н/Д,Алматы,Toyota City,Юр. Лицо,1.000000,"30870,28194","30870,28194",г.Алматы,Пикапы,Pick-ups
33070,2019,Июль,Toyota Motor Kazakhstan,Toyota,HILUX DC,2019,Таиланд,Бензин,#Н/Д,#Н/Д,#Н/Д,Атырау,Тойота Центр Атырау,Физ. Лицо,1.000000,"27012,4573","27012,4573",Атырауская область,Пикапы,Pick-ups


In [130]:
df.query('brend == "Toyota" & model == "HILUX DC"').sample(5)
df.query('brend == "Toyota" & model == "Corolla"').sample(5)

,year,month,company,brend,model,year_release,manufacturing_country,type_of_fuel,volume_engine,transmission,type_of_drive,region,diler,client_type,count,price,sale,area,segment_2013,class_2013
31821,2019,Июль,Toyota Motor Kazakhstan,Toyota,Corolla,2019,Турция,Бензин,2.7,AT,4WD,Алматы,Toyota City,Юр. Лицо,1.000000,"41664,14508","41664,14508",г.Алматы,Легковые автомобили,C класс
31195,2019,Март,Toyota Motor Kazakhstan,Toyota,Corolla,2 019,Турция,Бензин,2.7,AT,4WD,Атырау,Тойота Центр Атырау,Физ. Лицо,1.000000,"42413,7931","42413,7931",Атырауская область,Легковые автомобили,C класс
31851,2019,Июль,Toyota Motor Kazakhstan,Toyota,Corolla,2019,Турция,Бензин,2.7,AT,4WD,Алматы,Toyota City,Физ. Лицо,1.000000,"41664,14508","41664,14508",г.Алматы,Легковые автомобили,C класс
31532,2019,Июнь,Toyota Motor Kazakhstan,Toyota,Corolla,2019,Турция,Бензин,2.7,AT,4WD,Алматы,Тойота Центр Жетысу,Юр. Лицо,1.000000,"41856,70296","41856,70296",г.Алматы,Легковые автомобили,C класс
31781,2019,Июль,Toyota Motor Kazakhstan,Toyota,Corolla,2019,Турция,Бензин,2.7,AT,4WD,Нур-Султан,Toyota City Астана,Физ. Лицо,1.000000,"22282,38637","22282,38637",г.Нур-Султан,Легковые автомобили,C класс


In [131]:
df.loc[(df['volume_engine'] == '#Н/Д') & (df['model'] == "Corolla"), 'volume_engine'] = '1.6'
df.loc[(df['volume_engine'] == '#Н/Д') & (df['model'] == 'HILUX DC'), 'volume_engine'] = '2.7'

In [132]:
df.volume_engine.isna().mean()

0.045538707901716456

In [133]:
df = df.dropna(subset='volume_engine')

In [134]:
df.volume_engine.unique()

array(['2', '1,6', '1.4', '3', '4.4', '6.6', '1.5', '1.596', '2.4', '1.7',
       '26.7', '25.7', '24.7', '23.7', '22.7', '21.7', '20.7', '19.7',
       '18.7', '17.7', '16.7', '15.7', '14.7', '13.7', '12.7', '11.7',
       '10.7', '9.7', '8.7', '7.7', '6.7', '5.7', '4.7', '3.7', '2.7',
       '6.2', '2.5', '1.6', '2.8', '4.3', '2.0', '3.9', '7.5', '12.3',
       '3.5', '5.6', '5.2', '3/0', '5', '1.598', '1.248', '1.998',
       '2.359', '1.999', '3.342', '1.591', '3.47', '1.69', '1.774',
       '2.7 ', '4.6', '1.8', '10.5', '4', '5.5', '12', '12.8', '11',
       '2.2', '1.2', '1.485', '0', '13', '1.33', '4.0', '2.693', '3.8',
       '3.6', '7.6', '4.9', '2.9', '4.98', '8.4', '6.5'], dtype=object)

In [135]:
df.volume_engine.isna().sum()
df.volume_engine = df.volume_engine.replace({'1,6':'1.6','3/0':'3.0'})
df['volume_engine'] = df['volume_engine'].astype('str')
df['volume_engine'] = df['volume_engine'].astype('float')

#### Сount | Количество продаж

In [136]:
df['count'].isna().sum().mean()
# нет данных по объему реализации и типу клиента
df = df.dropna(subset='count')

# замена вещественного типа на целочисленный
df['count'] = df['count'].astype('float')
df['count'] = df['count'].astype('int')

#### Year , year_release | Год | Год выпуска

In [137]:
df.year_release.unique()

array(['2019', '2018', '2017', nan, '2016', '2014', '2013', '2\xa0018',
       '2\xa0019'], dtype=object)

In [138]:
# уникальные значения по критериям
int_64 = ['year_release','year']
for index in int_64:
    print(df[index].unique())

# проверка количества нулевых значений
df[df['year_release'].isna()]

df[df.year_release.isna()].model.value_counts().nlargest(10)
year_nan = df[df.year_release.isna()].model.value_counts().nlargest(10).reset_index()['index'].tolist()
df[df.model.isin(year_nan)].groupby('model')['year_release'].value_counts()

#заполнение пропусков года выпуска - модой года выпуска по моделям
df['year_release'] = df.groupby(['model'], sort = False)['year_release'].apply(lambda x: x.fillna(x.mode()[0])
                                                                               if len(x.mode()) >= 1 else x)

#работа с пропусками в количественной переменной /предобработаны скрытые пробелы \xa0
df['year_release'] = df ['year_release'].replace({'2\xa0019':'2019','2\xa0018':'2018'})
#замена оставшихся пропусков по бренду
df['year_release'] = df.groupby(['brend'], sort = False)['year_release'].apply(lambda x:x.fillna (x.mode()[0]))

df['year_release'] = df['year_release'].astype('int')
# проверка уникальных значений
df['year_release'].unique()

['2019' '2018' '2017' nan '2016' '2014' '2013' '2\xa0018' '2\xa0019']
[2019]


array([2019, 2018, 2017, 2016, 2014, 2013])

#### Price , sale | Цена , продажа (USD)

In [139]:
# предобработка
df[['price','sale']] = df[['price','sale']].replace(',', '.', regex=True)

df['price'] = pd.to_numeric(df['price'], errors='coerce').round(0)
df['sale'] = pd.to_numeric(df['sale'], errors='coerce').round(0)

# обработка по модулю
df['sale'] = df['sale'].abs()
df['count'] = df['count'].abs()

## ПРЕДОБРАБОТКА. Категориальные признаки


#### Month | Месяц

In [140]:
df.month.unique()

array(['Январь', 'Май', 'Август', 'Апрель', 'Июль', 'Февраль', 'Июнь',
       'Сентябрь', 'Март'], dtype=object)

In [141]:
# преобразование формата , типа, генерация даты
month_dict = {'Январь':1,'Февраль':2,'Март':3,'Апрель':4,'Май':5,
              'Июнь':6,'Июль':7,'Август':8,'Сентябрь':9}
df['month'] = df['month'].map(month_dict)
df['date'] = df['month'].astype('str') +"-"+ df ['year'].astype('str')

df['date'] = pd.to_datetime(df['date'],format='%m-%Y')

#### Client_type | Тип клиента

In [142]:
print(df.client_type.unique())
print(f'В признаке "тип клиента" {df.client_type.isna().sum()} пропусков')

df.groupby('client_type')['count'].max()

[nan 'Физ. Лицо' 'Юр. Лицо' 'Юр.Лицо' 'физ.лицо' 'юридическое'
 'физическое' 'ФизЛицо' 'ЮрЛицо' 'Корп. клиент']
В признаке "тип клиента" 6636 пропусков


client_type
Корп. клиент     1
Физ. Лицо        7
ФизЛицо          1
Юр. Лицо        70
Юр.Лицо          1
ЮрЛицо           1
физ.лицо         1
физическое       1
юридическое      1
Name: count, dtype: int64

In [146]:

# выделены 2 датасета для работы с пропусками в типе клиента
df_client_nan = df[df.client_type.isna()]
df_client =  df[~df.client_type.isna()]

# функция по взаимосвязи типа клиента с количеством покупок авто
# Допущения 2 - физ.лицо , более 2 - юр.лицо
def client_count(count):
    if count >=2.0:
        return 'юр.лицо'
    return "физ.лицо"
df_client_nan['client_type'] = df_client_nan['count'].apply(client_count)
# слияние датасетов
df = pd.concat ((df_client_nan,df_client))
df.shape

# функция для категоризации по типу клиенту
def type_client (index):
    if index in ['Физ. Лицо','физ.лицо','физическое','ФизЛицо','Корп. клиент']:
        return 'физ.лицо'
    return 'юр.лицо'

#проверим распределение категории
df['client_type'] = df['client_type'].apply(type_client)
print(df['client_type'].unique())
df.client_type.isna().sum()

['физ.лицо' 'юр.лицо']


0

#### Company , brend, manufacturing_country | Компания, бренд , страна производителя

In [147]:
# уникальные значения по критериям
int_64 = ['company','brend','manufacturing_country']
for index in int_64:
    print(df[index].unique())

['Renault Россия' 'Автоцентр-Бавария' 'БИПЭК АВТО' 'Вираж'
 'Hyundai Com Trans Kazakhstan ' 'Nissan Manufacturing RUS' 'СВС-ТРАНС'
 'Allur Auto' 'Almaty Motors Premium' 'MAN Truck & Bus Kazakhstan'
 'Autokapital' 'Автокапитал' 'ММС Рус' 'MMC RUS' 'УзАвто-Казахстан'
 'Volkswagen Group Rus' 'Scandinavian Motors' 'Mercur Auto'
 'Eurasia Motor Premium' 'Daewoo Bus Kazakhstan' 'Caspian Motors'
 'Каспиан Моторс' 'Astana Motors' 'Astana Motors ' 'TERRA MOTORS'
 'Toyota Motor Kazakhstan' 'Лифан Моторс Рус' 'Mercur Autos'
 'Равон Моторс Казахстан' 'Ravon Motors Kazakstan' 'Scania Central Asia'
 'Subaru Kazakhstan' 'Автодом Motors KST' 'Автомир-Центр'
 'ТОО "Eurasia Motor Zhaik"' 'Хино Моторс Казахстан' 'Hino Motors ']
['Renault' 'BMW' 'Chevrolet' 'GAZ' 'Hyundai Truck & Bus' 'Infiniti'
 'Isuzu' 'Jac' 'Jaguar' 'Kia' 'Lada' 'Land Rover' 'MAN' 'Mercedes-Benz'
 'Mercedes-Benz Trucks' 'Mini' 'Mitsubishi' 'Nissan' 'Ravon' 'Skoda' 'UAZ'
 'Volkswagen' 'Volvo' 'ANKAI' 'Audi' 'Daewoo' 'Ford' 'Hyundai' 'Iv

In [148]:
# предобработка пробелов по столбцам
a = ['company','brend']
for index in a:
    df[index] = df[index].str.strip()


# убираем неявные дубликаты в компаниях
df['company'] = df ['company'].replace({'Каспиан Моторс':'Caspian Motors',
                                        'ММС Рус':'MMC RUS','Mercur Autos':'Mercur Auto',
                                        'Автокапитал':'Autokapital','Mercur Autos':'Mercur Auto',
                                        'Хино Моторс Казахстан':'Hino Motors',
                                        'Равон Моторс Казахстан':'Ravon Motors Kazakstan'})

print('В названиях компании {} уникальных наименований'.format(df.company.nunique()))

# скрытых дубликатов не выявлено
print('В названиях бренда {} уникальных наименований'.format(df.brend.nunique()))

# замена наименований столбцов для отображения по коду страны
dict_country={'Германия':'DEU','США':'USA', 'Австрия':'AUT','Республика Казахстан':'KAZ','Российская Федерация':'RUS',
              'Корея':'KOR','Япония':'JAM','Таиланд':'THA','Китай':'CHN','UK':'UK',
              'Узбекистан':'UZB','Венгрия':'HUN','Турция':'TUR',
              'Испания':'ESP','Нидерланды':'NLD','Польша':'POL','Швеция':'SWE','Белоруссия':'BLR','Бельгия':'BEL'}

df['manufacturing_country'] = df['manufacturing_country'].map(dict_country)

В названиях компании 30 уникальных наименований
В названиях бренда 39 уникальных наименований


#### Type_of_fuel | Тип топлива

In [151]:
df.type_of_fuel.unique()
#df.type_of_fuel.isna().mean()

array([nan, 'Бензин', 'Дизель', '1,6', '0', 'гибрид', 'дизель ',
       'бензин '], dtype=object)

In [152]:
# проверка и корректировка значений тип топлива
df.query('type_of_fuel == "1,6" | type_of_fuel == "0"')

df.loc[(df['model'] == '3741') & (~df.type_of_fuel.isna())]
df.loc[(df['model'] == 'Sandero') & (~df.type_of_fuel.isna())]

,year,month,company,brend,model,year_release,manufacturing_country,type_of_fuel,volume_engine,transmission,type_of_drive,region,diler,client_type,count,price,sale,area,segment_2013,class_2013,date
24997,2019,1,Renault Россия,Renault,Sandero,2018,RUS,"1,6",0.000000,0,0,Атырау,Урал-Кров Авто,физ.лицо,1,12477.000000,12477.000000,Атырауская область,Легковые автомобили,B класс,2019-01-01
25002,2019,1,Renault Россия,Renault,Sandero,2018,RUS,"1,6",0.000000,0,0,Атырау,Урал-Кров Авто,физ.лицо,1,12477.000000,12477.000000,Атырауская область,Легковые автомобили,B класс,2019-01-01
25023,2019,1,Renault Россия,Renault,Sandero,2018,RUS,"1,6",0.000000,0,0,Нур-Султан,Автоцентр Астэк,физ.лицо,1,12477.000000,12477.000000,г.Нур-Султан,Легковые автомобили,B класс,2019-01-01
25231,2019,6,Renault Россия,Renault,Sandero,2018,RUS,Бензин,1.600000,MT,Передний,Алматы,ASTER AUTO Алматы,физ.лицо,1,12701.000000,12701.000000,г.Алматы,Легковые автомобили,B класс,2019-06-01


In [159]:
# тип топлива нижний регистр
df['type_of_fuel'] = df['type_of_fuel'].str.strip().str.lower()

#заполнение пропусков
df[df.type_of_fuel.isna()].model.value_counts().nlargest(10)
type_of_fuel_nan = df[df.type_of_fuel.isna()].model.value_counts().nlargest(10).reset_index()['index'].tolist()
type_of_fuel_nan

df[df.model.isin(type_of_fuel_nan)].groupby('model')['type_of_fuel'].value_counts()

df['type_of_fuel'] = df.groupby(
    ['model'], sort = False)['type_of_fuel'].apply(lambda x: x.fillna(x.mode()[0])
                                                   if len(x.mode()) >= 1 else x)

print(f'Оставшееся количество пропусков {df.type_of_fuel.isna().sum()}')

# принимаем решение удалить оставшиеся пропуски по признаку
#df = df.dropna(subset=['type_of_fuel'])
# поставим заглушки на пропуски
df ['type_of_fuel'] = df ['type_of_fuel'].fillna('не определено')


#создадим функцию category_fuel_type для распределения типов топлива на категории
def category_fuel_type(index):
    if index in ['бензин', '2', '1,6', '0']:
        return 'бензин'
    elif index in ['дизель']:
        return 'дизель'
    elif index in ['электро','электричество']:
        return 'электро'
    elif index in ['гибрид']:
        return 'гибрид'

    return 'не определено'

#проверим распределение категории
df['type_of_fuel'] = df['type_of_fuel'].apply(category_fuel_type)
df['type_of_fuel'].unique()
print(f'Оставшееся количество пропусков {df.type_of_fuel.isna().sum()}')


Оставшееся количество пропусков 0
Оставшееся количество пропусков 0


#### Transmission |  Коробка передач

In [161]:
#создадим функцию category_transmission для распределения коробки передач
m = ['5 МТ','MT','5МТ','Мех.','6 MT', '6MT','МКПП','МТ','6 МТ','6МТ','5MT','6M/T',
     'МT','5M','M/T', 'МКП', '5М', 'М/T']

def category_transmission(index):
    if index in m:
        return 'MT'
    return 'AT'

#проверим распределение категории
df['transmission'] = df['transmission'].apply(category_transmission)
df['transmission'].unique()

array(['MT', 'AT'], dtype=object)

#### Diler | Наименование дилерского центра

In [163]:
print('Уникальные названия дилерского центра составляют {} наименований'.format(df.diler.nunique()))

# Перечесение наименований по компании  и дилерскому центру
df[['diler','company']].sample(10)

Уникальные названия дилерского центра составляют 176 наименований


,diler,company
29375,Toyota City,Toyota Motor Kazakhstan
37458,БИПЭК АВТО Актау,БИПЭК АВТО
5157,Hyundai Auto Kostanai,Astana Motors
5151,Hyundai Center Shimkent,Astana Motors
20904,Eurasia Motor Атырау,Nissan Manufacturing RUS
34616,Тойота Центр Актау,Toyota Motor Kazakhstan
31690,Toyota City,Toyota Motor Kazakhstan
29501,Тойота Центр Актау,Toyota Motor Kazakhstan
37961,Mercur Auto Алматы,Mercur Auto
4297,Hyundai Center Petropavlovsk,Astana Motors


In [164]:
df = df.drop (columns = ['diler'])

#### Type_of_drive | Тип привода

In [165]:
df.type_of_drive.unique()

array(['4WD', 'Передний', 'Задний', 'Полный', 'AWD', nan, '4X4', '4х4',
       '4x4', '2WD', '4x2', '4х2', '4X2', 'RWD', 'FWD', '0', 'передний',
       'quattro', '4х2.2', '2 WD', '4 WD', 'Передний (FF)', 'FF', '#Н/Д',
       'полный', '4Motion'], dtype=object)

In [166]:
#создадим функцию categorize_drive_type для распределения типов привода на категории
def categorize_drive_type(cell):
    if cell in ['передний', 'FWD', 'Передний', 'Передний (FF)','FF']:
        return 'FWD'   #передний привод
    elif cell in ['4WD','quattro', 'Полный', 'полный', 'AWD', '4 WD', '4Motion']:
        return '4WD'   #полный привод
    elif cell in ['Задний','RWD']:
        return 'RWD' #задний привод
    return '2WD' #другие

df['type_of_drive'] = df['type_of_drive'].apply(categorize_drive_type)

#### Area, segment, class

In [168]:
df.area.unique()

array(['Западно-Казахстанская область', 'г.Нур-Султан',
       'Кызылординская область', 'Карагандинская область',
       'Восточно-Казахстанская область', 'Южно-Казахстанская область',
       'г.Алматы', 'Костанайская область', 'Северо-Казахстанская область',
       'Павлодарская область', 'Мангистауская область',
       'Атырауская область', 'Актюбинская область', 'Акмолинская область',
       'Жамбылская область', 'Экспорт область', 'Алматинская область',
       'Туркестанская область'], dtype=object)

In [169]:
df.segment_2013.unique()

array(['Внедорожники', 'Легковые автомобили', 'Коммерческие автомобили',
       'Пикапы', 'Минивэны'], dtype=object)

In [170]:
df.class_2013.unique()

array(['Субкомпактные SUV', 'B класс', 'D класс', 'E класс',
       'Спортивные автомобили', 'F класс', 'Компактные SUV',
       'Среднеразмерные SUV', 'Полноразмерные SUV',
       'Малотоннажные грузовики', 'Микроавтобусы',
       'Среднетоннажные грузовики', 'Крупнотоннажные грузовики',
       'Большие автобусы', 'Pick-ups', 'C класс', 'A класс', 'Компактвэн',
       'Развозные автомобили', 'Полноразмерный Минивэн',
       'Средние автобусы'], dtype=object)

In [ ]:
# сохраняю предобработанный датасет
#df.to_csv('C:/Users/vdavy/Desktop/pet/df.csv', header=False, index=False)

OSError: Cannot save file into a non-existent directory: 'C:/Users/vdavy/Desktop/pet'

In [ ]:
#from google.colab import drive
#drive.mount("/content/drive")
#!pwd
#!ls /content/drive/MyDrive/auto_kz_2019_датасет.csv
#!ls /content/drive/MyDrive/collab/df.csv

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#df.to_csv('df_1.csv')
#!cp df_1.csv "/content/drive/MyDrive"

Mounted at /content/drive


###  ИТОГИ ПРЕДОБРАБОТКИ

1) «Месяц» --- Исходные данные с Января по Сентябрь (неполный зимний сезон ( Январь - Февраль ) , весенний сезон , летний сезон, начало осеннего сезона (Сентябрь). Большинство данных содержат весенне-летний сезон. Учитываем при анализе и рекомендациях в стратегии компании.

2) «Компания» --- Для верных суммарных значений в рамках одной компании, предобработка названий одной компании на разных языках (скрытые дубликаты). Уникальных названий компании - 33 .

3) «Бренд» --- Предобработаны лишние пробелы в названиях,  в данном признаке представлено 52 бренда компании производителей авто.Неявных дубликатов не обнаружено.

4) «Модель» --- Предобработаны лишние пробелы , в данном признаке  представлено 273 уникальных наименования моделей авто

5) «Модификация» --- В данном признаке  представлено 1319  уникальных наименования модификации. Пропуски 3373 ,  8 % от общего числа. Удаляем данный столбец , данный признак не критичен в анализе.

6) «Cтрана производитель» --- Предобработан формат - отображение по коду стран , использован внешний источник Alpha 3

7) «Вид топлива» --- Предобработка формата, присутствуют значения из других столбцов (невнимательность при заполнении таблицы). Значения '2', '1,6' встречаются у моделей Renault Duster
Значения '0' UAZ 3741 модификация 3741. При мониторинге информации из внешних источников + анализ датасете - замена на тип двигателя "бензин".
7,8 % пропусков. Пропуски заполнены модой по модели, там, где есть доминирование того или иного вида топлива.

8) «Коробка передач»  --- Категоризируем признак на 2 типа коробки переключения передач:
 - автоматическая (АТ)
 - механическая (МТ)
Допущения : роботизированные коробки передач, редукторы, DSG и различные вариации tronic будем считать для удобства автоматической коробкой передач.

9) «Регион» ---  Уникальных названий 26 . Допущения - названия городов Республики Казахстан

10) -11) «Сегмент 2013» и «Класс 2013» – категориальные. Уточнение актуальности этих данных

12 - 14) «Сегмент», «Форма расчета», «Сегмент Eng» --- можно удалить (множество незаполняемых пропусков или дублируются другими столбцами)

15) «Наименование дилерского центра» ---  пересечение с названиями компаний. Возможно удалить данный признак

16) «Тип клиента» - предобработан к единому формату , категоризация на 2 типа : физическое лицо и юридическое лицо. Предобработка пропущенных значений, корреляция с количеством реализаций

17) «Объем двигателя»
- Количество моделей составляет 38 строк, где в объеме двигателя указаны значения #Н/Д, MT, AT
(MT, AT - коробки передач , гипотеза : смещение данных ). Данное смещение устранено.
- 400 Л.С. согласно калькулятору составляет 2 литра, замена на 2.0
- пропуски заполнены модой по модели , так же как пропуски в признаке "Тип топлива"
Принято решение удалить все буквенные символы, оставить только числа, перевести в количественную переменную.





- Количество строк до обработки составляло 39966
- Количество  строк после обработки составило 38410

   Фиксируется процент уменьшения строк в 4 % , допустимая погрешность
